for dataset :
https://www.kaggle.com/datasets/algord/fake-news

In [87]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences


In [88]:
df = pd.read_csv("FakeNewsNet.csv")
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [89]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          23196 non-null  object
 1   news_url       22866 non-null  object
 2   source_domain  22866 non-null  object
 3   tweet_num      23196 non-null  int64 
 4   real           23196 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 906.2+ KB


In [90]:
df = df.dropna()
df.shape

(22866, 5)

In [91]:
y = df["real"]
y.value_counts()

1    17371
0     5495
Name: real, dtype: int64

In [92]:
df.drop("real", axis = 1, inplace =True)
data_copy = df.copy()
data_copy.head()

<ipython-input-92-afa58bba4071>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("real", axis = 1, inplace =True)


,title,news_url,source_domain,tweet_num
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38


In [93]:
x = df["title"]
x.shape

(22866,)

In [94]:
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [95]:
from nltk.corpus import stopwords
ps = PorterStemmer()
import re
corpus = []
for i in x:
  a = re.sub("[^a-zA-Z ]","", i)
  a = a.lower()
  a = a.split()

  a =" ".join([ps.stem(w) for w in a if not w in stopwords.words('english')])
  corpus.append(a)


In [96]:
corpus[0:6]

['kandi burruss explod rape accus real housew atlanta reunion video',
 'peopl choic award best red carpet look',
 'sophia bush send sweet birthday messag one tree hill costar hilari burton breyton eva',
 'colombian singer maluma spark rumour inappropri relationship aunt',
 'gossip girl year later upper east sider shock world chang pop cultur forev',
 'gwen stefani got dump blake shelton jealousi drama exclus']

In [97]:
voc_size=20000
one_hot_data = [one_hot(sen, voc_size,) for sen in corpus]
one_hot_data[0:5]

[[18628, 13034, 14492, 14179, 15178, 11335, 1896, 13319, 17583, 2140],
 [9654, 13724, 16308, 14705, 6503, 1930, 7172],
 [2228,
  13865,
  9493,
  17574,
  304,
  11773,
  8779,
  13461,
  18264,
  6589,
  3026,
  10570,
  4510,
  1766],
 [12478, 3037, 16639, 3233, 11942, 1681, 3473, 18156],
 [7725,
  5984,
  9674,
  8753,
  15855,
  13806,
  11198,
  7061,
  7771,
  4563,
  3219,
  1791,
  8626]]

In [98]:
sen_len = 25
pad_data = pad_sequences(one_hot_data, padding = "pre", maxlen=sen_len)
pad_data[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0, 18628, 13034, 14492,
       14179, 15178, 11335,  1896, 13319, 17583,  2140], dtype=int32)

In [99]:
pad_data = np.array(pad_data)
y = np.array(y)
pad_data.shape, y.shape

((22866, 25), (22866,))

In [100]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pad_data, y, test_size=0.23, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((17606, 25), (5260, 25), (17606,), (5260,))

In [101]:
dim = 40
model = Sequential()
model.add(Embedding(voc_size , dim, input_length= sen_len ))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = "sigmoid"))
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics= "accuracy")
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 25, 40)            800000    
                                                                 
 dropout_12 (Dropout)        (None, 25, 40)            0         
                                                                 
 lstm_6 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_13 (Dropout)        (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 856501 (3.27 MB)
Trainable params: 856501 (3.27 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [102]:
model.fit(X_train,y_train, epochs =10, validation_data =(X_test,y_test), batch_size =32)

Epoch 1/10
551/551 [==============================] - 14s 21ms/step - loss: 0.4449 - accuracy: 0.8066 - val_loss: 0.3777 - val_accuracy: 0.8395
Epoch 2/10
551/551 [==============================] - 5s 9ms/step - loss: 0.3273 - accuracy: 0.8593 - val_loss: 0.3794 - val_accuracy: 0.8426
Epoch 3/10
551/551 [==============================] - 4s 6ms/step - loss: 0.2744 - accuracy: 0.8841 - val_loss: 0.3920 - val_accuracy: 0.8291
Epoch 4/10
551/551 [==============================] - 4s 8ms/step - loss: 0.2318 - accuracy: 0.9043 - val_loss: 0.4464 - val_accuracy: 0.8287
Epoch 5/10
551/551 [==============================] - 4s 7ms/step - loss: 0.2027 - accuracy: 0.9180 - val_loss: 0.4667 - val_accuracy: 0.8209
Epoch 6/10
551/551 [==============================] - 4s 6ms/step - loss: 0.1788 - accuracy: 0.9289 - val_loss: 0.5002 - val_accuracy: 0.8144
Epoch 7/10
551/551 [==============================] - 4s 6ms/step - loss: 0.1579 - accuracy: 0.9379 - val_loss: 0.5536 - val_accuracy: 0.8015
Epoc

In [111]:
from sklearn.metrics import accuracy_score, confusion_matrix
pred = model.predict(X_test)
pred

165/165 [==============================] - 0s 2ms/step


array([[0.04641596],
       [0.99654335],
       [0.9999145 ],
       ...,
       [0.01756085],
       [0.9999944 ],
       [0.8262195 ]], dtype=float32)

In [106]:
# import tensorflow as tf
# pred = tf.argmax(pred, axis=1)
# pred

<tf.Tensor: shape=(5260,), dtype=int64, numpy=array([0, 0, 0, ..., 0, 0, 0])>

In [130]:
new_pred = []
for elements in pred:
  if elements[0] >= 0.5:
    new_pred.append([1])
  else:
    new_pred.append([0])

In [132]:
confusion_matrix(y_test,new_pred)

array([[ 810,  462],
       [ 595, 3393]])

In [134]:
accuracy_score(new_pred,y_test)

0.7990494296577947